In [1]:
#!pip install pycbc lalsuite ligo-common 
#!pip install gwpy

#run previous lines if install -q code doesnt work
! pip install -q 'lalsuite==6.66' 'bilby==0.6.1' 'gwpy==1.0.1'

     |████████████████████████████████| 28.5 MB 67 kB/s 
     |████████████████████████████████| 1.5 MB 39.6 MB/s 
     |████████████████████████████████| 1.4 MB 40.2 MB/s 
     |████████████████████████████████| 52 kB 1.0 MB/s 
     |████████████████████████████████| 1.6 MB 18.4 MB/s 
     |████████████████████████████████| 87 kB 5.9 MB/s 
     |████████████████████████████████| 45 kB 3.5 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 3.2 MB 34.1 MB/s 


In [2]:
import gwpy
from gwpy.timeseries import TimeSeries

In [ ]:
#uncomment and run this block of code instead if import gwpy and TimeSeries doesnt work
'''from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt

import bilby
from bilby.core.prior import Uniform
from bilby.gw.conversion import convert_to_lal_binary_black_hole_parameters, generate_all_bbh_parameters

from gwpy.timeseries import TimeSeries''' 

In [ ]:
! git clone https://github.com/matteobreschi/bajes.git

In [ ]:
%cd bajes
! ls
! python setup.py install

In [ ]:
#necessary to run TEOBResumS_NRPM approx but not necessary if using only NRPM
! git clone https://RoxGamba@bitbucket.org/eob_ihes/teobresums.git

In [ ]:
! apt update
! apt install build-essential
! apt-get install -y libconfig-dev
!apt-get install libgsl-dev
% cd teobresums/Python/
!python TEOBResumSWrap_setup.py install

In [7]:
from __future__ import division, print_function
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from bajes.bajes.obs.gw import Series
from bajes.bajes.obs.gw import Noise
from bajes.bajes.obs.gw.utils import read_asd
import csv

In [ ]:
#import drive so csv file wont just save during runtime
from google.colab import drive 
drive.mount('/content/drive/', force_remount = True)

In [ ]:
time_of_event = 1126259462.4

post_trigger_duration =4
duration = 8
analysis_start = time_of_event + post_trigger_duration - duration

# Use gwpy to fetch the open data
H1_analysis_data = TimeSeries.fetch_open_data(
    "H1", analysis_start, analysis_start + duration, sample_rate=4096, cache=True)

t = H1_analysis_data.times
strain = H1_analysis_data.value

# set the data properties coherently
seglen = 8           # duration of the segment [s]
srate  = 4096         # sampling rate [Hz]
t_gps  = 0  # central value of GPS time
f_max  = 1024
f_min  = 20 

series = Series('time', strain, seglen=seglen, srate=srate, t_gps=t_gps, f_min=f_min, f_max=f_max)
print('here')
from bajes.bajes.obs.gw import Noise
from bajes.bajes.obs.gw.utils import read_asd

fr_asd, asd = read_asd('GW150914', 'H1')
noise       = Noise(fr_asd, asd, f_min=0, f_max=f_max)

from bajes.bajes.obs.gw import Detector, Waveform

wave  = Waveform(series.freqs, srate, seglen, 'TEOBResumS_NRPM') #NRPM for postmergers

In [ ]:

for num in range(20000):
  from scipy.interpolate import interpolate as interp

  eosdir = '/content/drive/MyDrive/Colab_Notebooks/macro'
  numeos = 2396


  def get_mmax(eos_id):
    eospath = eosdir+"/macro-spec_%dcr.csv" % eos_id
    eos_data = np.genfromtxt(eospath, names=True, delimiter=",")
    marray =  eos_data["M"]
    mmax_pos   = np.argmax(marray)
    mmax = marray[mmax_pos]
    return mmax

  def get_lambda_from_masses(m1, m2, eos_id):

    eospath = eosdir+"/macro-spec_%dcr.csv" % eos_id
    eos_data = np.genfromtxt(eospath, names=True, delimiter=",")
    marray, larray =  eos_data["M"], eos_data["Lambda"]
    mmax_pos   = np.argmax(marray)
    marray, larray = marray[0:mmax_pos+1], larray[0:mmax_pos+1]
    mmax = marray[mmax_pos]

    assert m1 <= mmax and m2 <= mmax, "Error: one or both masses too heavy for a NS!"

    f_lambda = interp.interp1d(marray, larray, fill_value=0, bounds_error=False)
    lambda1, lambda2 = f_lambda(m1), f_lambda(m2)

    assert lambda2 >= lambda1, "Error: Lambda1 is larger than Lambda2 for common EOS!"

    return lambda1, lambda2

  
  import math
  import random
  eos_id = np.random.choice(range(numeos), 1)
  mmax = get_mmax(eos_id)

  m1 = random.uniform(1, mmax) 
  m2 = random.uniform(1, mmax)

  # m1 has to be bigger than m2 for the proper mass ratio 
  if m1 > m2:
    chirp_mass = (math.pow((m1 * m2), (3/5))) / (math.pow((m1 + m2), (1/5))) 
    q = m1 / m2  #calculates the mass ratio
  elif m1 < m2:
    while m1 < m2:
      m1 = random.uniform(1, mmax)
      m2 = random.uniform(1, mmax)
  chirp_mass = (math.pow((m1 * m2), (3/5))) / (math.pow((m1 + m2), (1/5)))
  q = m1 / m2

  lambda1, lambda2 = get_lambda_from_masses(m1, m2, eos_id)

  params = {'mchirp'       : chirp_mass,    # chirp mass [solar masses] 
              'q'          : q,      # mass ratio 
              's1x'        : 0.,      # primary spin parameter, x component
              's1y'        : 0.,      # primary spin parameter, y component
              's1z'        : 0.,      # primary spin parameter, z component
              's2x'        : 0.,      # secondary spin parameter, x component
              's2y'        : 0.,      # secondary spin parameter, y component
              's2z'        : 0.,      # secondary spin parameter, z component
              'lambda1'    : lambda1,    # primary tidal parameter 
              'lambda2'    : lambda2,    # secondary tidal parameter
              'distance'   : 100.8114416513031,    # distance [Mpc]   
              'iota'       : np.pi,   # inclination [rad]   
              'ra'         : 0.,     # right ascension [rad]
              'dec'        : 0.,   # declination [rad]
              'psi'        : 0.,      # polarization angle [rad]
              'time_shift' : 0.419,   # time shift from GPS time [s]
              'phi_ref'    : 0.,      # phase shift [rad]
              'f_min'      : 20.,     # minimum frequency [Hz]
              'srate'      : srate,   # sampling rate [Hz]
              'seglen'     : seglen,  # segment duration [s] 
              'tukey'      : 0.1,     # parameter for tukey window
              't_gps'      : t_gps,   # GPS trigger time
              'lmax'       : 0.,
              'eccentricity' : 0.
             }  

  hpc = wave.compute_hphc(params)


  new_params = params #put params in new dictionary
  new_params["m1"] = m1 #add m1 to new dictionary
  new_params["m2"] = m2 #add m2 to new dictionary
  new_params["mtot"] = round((m1 + m2), 1) #change mtot from params into a float with only 1 decimal point


 #the very first run of the code, next_simulation is set at 1
 #if eos code throws an error while being iterated, change the next_simulation variable to the last simulation number + 1 so if 
 #last simulation file name number is S00051 then in the next_simulation variable, put the number 52
  next_simulation = 1
  if num == 0:
    new_num = num + next_simulation
  else:
    new_num = new_num + 1

  #changes the file name for each sim run
  if new_num <= 9:
    leading_zeros = "S0000"
  elif new_num <= 99:
    leading_zeros = "S000"
  elif new_num <= 999:
    leading_zeros = "S00"
  elif new_num <= 9999:
    leading_zeros = "S0"
  else:
    leading_zeros = "S"

  new_params["file name"] = f"{leading_zeros}{new_num}.csv"
  text_file = f'/content/drive/MyDrive/Colab_Notebooks/time_series/new_sims/{leading_zeros}{new_num}.csv'

  #separates the time series into two columns so it can be in a dataframe
  x = series.times
  y = hpc.plus
  ts_df= pd.DataFrame({'time': x, 'strain': y})

  #deletes rows from original dataframe so the time series csv will only be between -0.6 and +0.6 seconds
  ts_df.drop(ts_df.index[0:16138], 0, inplace = True)
  ts_df.drop(ts_df.index[493:], 0, inplace = True)
  #formats dataframe
  ts_df.to_csv(text_file, sep= '\t', header = None, index = False) 

  #creates original metadata file then appends the subsequent iterations
  if new_num == 1:
    df = pd.DataFrame.from_dict([new_params])
    df.to_csv('/content/drive/MyDrive/Colab_Notebooks/metadata_new.csv', sep= '\t', index = False)
  else: 
    from csv import DictWriter

    def append_dict_as_row(file_name, dict_of_elem, field_names):
      # Open file in append mode
        with open(file_name, 'a+', newline='') as write_obj:
          # Create a writer object from csv module
            dict_writer = DictWriter(write_obj, delimiter = '\t', fieldnames=field_names)
            # Add dictionary as row in the csv
            dict_writer.writerow(dict_of_elem)

    field_names = ['mchirp', 'q', 's1x', 's1y', 's1z', 's2x', 's2y', 's2z', 
               'lambda1', 'lambda2', 'distance','iota', 'ra', 'dec', 'psi','time_shift',
               'phi_ref', 'f_min', 'srate', 'seglen', 'tukey', 't_gps', 'lmax', 'eccentricity', 'm1', 'm2', 'mtot', 'file name'
               ]  

    row_dict = new_params
    # Append a dict as a row in csv file
    append_dict_as_row('/content/drive/MyDrive/Colab_Notebooks/metadata_new.csv', row_dict, field_names)





